# Data Ingest of HRRR weather model data

### Intro

Weather data predictors for the ML models of FMC are retrieved from the HRRR weather model in this project. The 3D pressure model product from HRRR is utilized, since it has a larger set of variables than other products and it is used internally in other areas of the `wrfxpy` project. Additionally, since we require rainfall for modeling, we utilize the 3-hour forecast from HRRR and use the difference in accumulated precipitation from the 2 to 3 hour forecasts.

There are 2 main uses for the HRRR weather data:

1. For constructing training data sets
2. For forecasting with a trained model over a spatial domain

This notebook will demonstrate reading and calculating a set of predictors derived from the HRRR model for a spatial bounding box.

### Metadata File

The metadata file `../etc/variable_metadata/hrrr_metadata.yaml` has information about how to construct various predictors of FMC from HRRR grib file data. There are 4 types of features used in this project: HRRR modeled variables (e.g. wind speed), HRRR dimension variables (e.g. time), features engineered from HRRR modeled data (e.g. equilibrium moisture), and features engineered from HRRR dimension variables (e.g. hour of day). These 4 types of features must be extracted and constructed differently. Top level keys in the metadata file are fmda names used within this project:

- HRRR data variables will specify HRRR naming convention, regex search string, and layer/level. Common layers are grouped together in data retrieval
- HRRR dimension variables will specify a HRRR naming convention, but they can be read from any other set of HRRR data
- Engineered features from HRRR data variables will specify the names of variables needed to calculate them. The names will exist as other top-level keys in this file
- Engineered features from HRRR dimension variables will specify the names of the dimensions needed to calculate them

### Code

A configuration file is used to control data ingest. For automated processes, the code will look for a json configuration file depending on the use case: 

* For building training data, `../etc/training_data_config.json`
* For deploying the model on a grid, `../etc/forecast_config.json`

Retrieval of atmospheric weather predictors is done with the python software package `Herbie`. A module `retrieve_hrrr_api.py` has functions and other metadata for directing data ingest. A list of predictors will be provided in order to control the data downloading. Some of these predictors are derived features, such as equilibrium moisture content which is calculated from relative humidity and air temperature. 

## References

For more info on HRRR data bands and definitions, see [HRRR inventory](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf02.grib2.shtml) for pressure model f02-f38 forecast hours.

For more info on python package, see Brian Blaylock's `Herbie` [python package](https://github.com/blaylockbk/Herbie)

## Setup

User definitions, these will come from config files in other areas of this project.

In [ ]:
import matplotlib.pyplot as plt
from herbie import FastHerbie, paint, wgrib2, Herbie
# from herbie.toolbox import EasyMap, ccrs, pc
import xarray as xr
from datetime import datetime
import sys
import pandas as pd
import numpy as np
sys.path.append("../src")
from utils import Dict, read_yml, str2time, print_dict_summary
import ingest.retrieve_hrrr_api as ih

In [ ]:
with open("../etc/training_data_config.json", "r") as json_file:
    config = json.load(json_file)   
    config = Dict(config)

bbox = config.bbox
start = str2time(config.start_time)
end = str2time(config.end_time)
features_list = config.features_list

print(f"Start Date of retrieval: {start}")
print(f"End Date of retrieval: {end}")
print(f"Spatial Domain: {bbox}")
print(f"Required Features: {features_list}")

### Retrieve Data

This function from `herbie` sets up a connection to read, but only what is requested later will be downloaded. Available data can be viewed with the `inventory()` method. *Note:* this will display a separate row for each time step requested.

In [ ]:
# Create a range of dates
dates = pd.date_range(
    start = start.replace(tzinfo=None),
    end = end.replace(tzinfo=None),
    freq="1h"
)

In [ ]:
FH = FastHerbie(
    dates, 
    model="hrrr", 
    product="prs",
    fxx=range(3, 4)
)

In [ ]:
inv = FH.inventory()
inv

### Spatial Subset

NOTE: as of Dec 31 2024, there are package issues with this solution. Herbie environment doesn't work either. TODO

Brian Blaylock recommends downloaded the data and spatially subsetting using Herbie's wrapper for `wgrib2`, then recreating the objects and reading into memory.

In [ ]:
# bbox

In [ ]:
# def get_fh_layer(FH, search_string, remove_grib=True, bbox=None, subset_naming="myRegion"):
#     """
#     Get HRRR data from fastherbie object given regex search string. 
#     Search string groups variables by layer/level. 
#     Optional bounding box spatially subsets data

#     Arguments:
#         - FH: FastHerbie object, defined with start and stop times
#         - remove_grib: bool, whether or not to delete grib files returning to local read
#         - search_string: str, based on regex. see utility function features_to_searchstr
#         - bbox: list, optional bounding box to subset region

#     Notes: As of Dec 18, 2024, Brian Blaylock recommends downloading data and using 
#         wgrib2 to spatially subset the data
        
#     Returns:
#         xarray, optionally subsetted to a bounding box
#     """

#     if bbox is None:
#         print("Returning data for entire conus, deleting all downloaded gribs")
#         ds = FH.xarray(search_string, remove_grib=remove_grib)
#     else:
#         print(f"Subsetting data to region within bbox: {bbox}")
#         print(f"Downloading Data to run wgrib2")

#         files = FH.download(search_string)
#         files = sorted(files, key=lambda x: int(x.name.split('__hrrr.t')[1][:2])) # sort by hour
        
#         # Reorder bbox to match format (min_lon, max_lon, min_lat, max_lat)
#         extent = (bbox[1], bbox[3], bbox[0], bbox[2]) 
#         subset_files=[]
#         for file in files:
#             subset_files.append(wgrib2.region(file, extent, name=subset_naming))

#         # Convert PosixPath list to strings
#         file_list = [str(path) for path in subset_files]
        
#         # Open files as a combined dataset
#         ds = xr.open_mfdataset(
#             file_list,
#             engine="cfgrib",
#             concat_dim="time",  # Replace 'time' with the appropriate dimension
#             combine="nested" 
#         )        
#         ds = ds.sortby('time')  

#         # Delete Files
#         if remove_grib:
#             for file in files:
#                 if file.exists():  # Check if the file exists before attempting to delete it
#                     file.unlink()        
#             for file in subset_files:
#                 if file.exists():  # Check if the file exists before attempting to delete it
#                     file.unlink()    
                
#     return ds

In [ ]:
# ss = search_strings['2m']

# ds1 = get_fh_layer(FH, ss)

In [ ]:
# ds2 = get_fh_layer(FH, ss, remove_grib=False, bbox = bbox)

In [ ]:
# # Get CRS from geographic herbie 
# ## Assuming this info doesn't change over time
# H = Herbie("2023-08-01", product="sfc")
# ds_hgt = H.xarray("(?:HGT|LAND):surface")
# crs = ds_hgt.herbie.crs

In [ ]:
# from herbie.toolbox import EasyMap

In [ ]:
# ax = EasyMap(crs=crs).STATES(color="k").ax
# ax.pcolormesh(ds_hgt.longitude, ds_hgt.latitude, ds_hgt.orog, cmap=paint.LandGreen.cmap, alpha=0.5, transform=pc)
# ax.pcolormesh(ds2.longitude, ds2.latitude, ds2.t2m.isel(time=0), transform=pc)

# ax.gridlines(xlocs=extent[:2], ylocs=extent[2:], color="k", ls="--", draw_labels=True)

Data fields are accessed through the `.xarray()` method. This will temporarily download the file and then deliver it in memory as an xarray object. Different variables are accessed through search strings that specify the variable name (e.g. air temperature), the level of the observation (e.g. surface level), and the forecast hour relative to the f00 start time (e.g. hour 3 as we will be using). The `retrieve_hrrr_api` module in this project stores a dataframe with names and info on various variables that will be considered for modeling FMC.

In [ ]:
# # Show HRRR naming dataframe
# ih.hrrr_name_df

## Read Data from Multiple Layers

We will demonstrate retrieval of a restricted set of predictors. Grib data is grouped by layer, or "hypercube". There are various options for combining data from different levels. See [Brian's solution](https://herbie.readthedocs.io/en/2024.3.1/user_guide/_bonus_notebooks/multiple_variables_and_merge.html) in the Herbie docs. In this project, we implement our own version.

The steps are:

- From a list of features with fmda names, or standard names used in this project, get a list of regex search strings for accessing HRRR grib files. These strings will be grouped by layer for easy reading with `FastHerbie`
- Read each separate layer of data and group, adding a dimension for height above ground.

In [ ]:
import importlib
import ingest.retrieve_hrrr_api
importlib.reload(ingest.retrieve_hrrr_api)
import ingest.retrieve_hrrr_api as ih

print(f"Target Features List: {features_list}")
search_strings = ih.features_to_searchstr(features_list)
print("HRRR Search Strings:")
print_dict_summary(search_strings)

In [ ]:
ds_dict = {}

for layer in search_strings:
    print(f"Reading HRRR data for layer: {layer}")
    print(f"    search strings: {search_strings[layer]}")
    ds_dict[layer] = FH.xarray(search_strings[layer], remove_grib=False) # Keep grib for easier re-use, delete later

In [ ]:
ds_dict.keys()

In [ ]:
ds_dict["surface"]

In [ ]:
ds_dict["2m"]

In [ ]:
ds_dict["10m"]

In [ ]:
def merge_datasets(ds_dict):
    """Merge list of Datasets together. Modified from Brian Blaylocks Herbie docs

    Since cfgrib doesn't merge data in different "hypercubes", we will
    do the merge ourselves.

    Parameters
    ----------
    ds_dict : dict
        A dictionary of xarray.Datasets organized by layer.

    Returns
    ----------
    ds : xarray DataSet
        Merged dataset with height above ground dimension added
    """


    # Check that all datasets in the dictionary have the same sizes
    sizes_list = [ds.sizes for ds in ds_dict.values()]
    if not all(sizes == sizes_list[0] for sizes in sizes_list):
        raise ValueError("All datasets must have the same .sizes attribute. Found mismatched dimensions")
    
    # Drop heightAboveGround coordinate if it exists
    for key, ds in ds_dict.items():
        if "heightAboveGround" in ds.coords:
            ds_dict[key] = ds.drop_vars("heightAboveGround")

    ds = xr.merge(ds_dict.values())
    return ds


In [ ]:
ds = merge_datasets(ds_dict)

In [ ]:
ds

In [ ]:
features_list

In [ ]:
hrrr_meta = ih.hrrr_meta

def rename_ds(ds):
    """
    Renames variables in an xarray Dataset based on the hrrr_meta dictionary.
    
    Parameters:
        ds (xr.Dataset): The input xarray Dataset.
        hrrr_meta (dict): Metadata dictionary containing xarray variable names and their new keys.
    
    Returns:
        xr.Dataset: Dataset with variables renamed.
    """
    rename_dict = {
        v['xarray_name']: key
        for key, v in hrrr_meta.items()
        if 'xarray_name' in v and v['xarray_name'] in ds
    }
    return ds.rename(rename_dict)

In [ ]:
rename_ds(ds)

## Construct Predictors

Equilibrium moisture content is calculated from RH and air temp.

In [ ]:
from ingest.retrieve_hrrr_api import calc_eq
calc_eq(ds)

In [ ]:
'time' in ds.dims

In [ ]:
inv = FH.inventory()

In [ ]:
inv[inv.variable == "WIND"]

In [ ]:
ds = FH.xarray("CNWAT:surface|ASNOW:surface|:SNOD:surface:3 hour fcst")

In [ ]:
ds

In [ ]:
ds.herbie.pick_points(
    pd.DataFrame({"latitude": [40.76, 40], "longitude": [-111.876183, -111]})
)

In [ ]:
type(ds)

In [ ]:
from herbie.toolbox import EasyMap, pc, ccrs
from herbie import paint

ax = EasyMap("110m", figsize=[15, 9], crs=ds.herbie.crs).STATES().ax

x = ds["Ed"]
if 'time' in x.dims:
    x = x.isel(time=4)

p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    x,
    transform=pc,
    cmap=paint.NWSRelativeHumidity.cmap,
)

plt.colorbar(
    p,
    ax=ax,
    orientation="horizontal",
    pad=0.01,
    shrink=0.8,
    label="Equilibrium Moisture Content",
)
plt.title(None, size=18)

In [ ]:
paint.NWSWindSpeed

In [ ]:
ax = EasyMap("50m", figsize=[15, 9], crs=ds.herbie.crs).STATES().ax
p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds.t2m.isel(time=0),
    transform=pc,
    cmap=None,
)

plt.colorbar(
    p,
    ax=ax,
    orientation="horizontal",
    pad=0.01,
    shrink=0.8,
    label="Equilibrium Moisture Content",
)

In [ ]:
ds = FH.xarray(":TSOIL:")
ds

In [ ]:
ds2 = FH.xarray(":APCP:surface:2-3 hour acc fcst")
ds2

In [ ]:
ax = EasyMap("50m", figsize=[15, 9], crs=ds.herbie.crs).STATES().ax
p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds2.tp.isel(time=0),
    transform=pc,
    cmap=paint.NWSPrecipitation.cmap,
)

plt.colorbar(
    p,
    ax=ax,
    orientation="horizontal",
    pad=0.01,
    shrink=0.8,
    label="Equilibrium Moisture Content",
)